## Exercise: Train a Model and Create Predictions

Train a model using the Boston dataset and a different set of input features.  Predict on new data.

In [0]:
%run "./Includes/Classroom-Setup"

Initialized classroom variables & functions...

Datasets are already mounted to /mnt/training from wasbs://training@dbtrainwesteurope.blob.core.windows.net/

Created user-specific database

Using the database saifahmed_k_outlook_com_db .

All done!

Import the Boston dataset, which contains median house values in 1000's (`medv`) for a variety of different features.  Since this dataset is "supervised" my the median value, this is a supervised machine learning use case.

In [0]:
bostonDF = (spark.read
  .option("HEADER", True)
  .option("inferSchema", True)
  .csv("/mnt/training/bostonhousing/bostonhousing/bostonhousing.csv")
)

display(bostonDF)

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
<command-510601638879694> in <cell line: 1>()
----> 1 bostonDF = (spark.read
      2   .option("HEADER", True)
      3   .option("inferSchema", True)
      4   .csv("/mnt/training/bostonhousing/bostonhousing/bostonhousing.csv")
      5 )

/databricks/spark/python/pyspark/instrumentation_utils.py in wrapper(*args, **kwargs)
     46             start = time.perf_counter()
     47             try:
---> 48                 res = func(*args, **kwargs)
     49                 logger.log_success(
     50                     module_name, class_name, function_name, time.perf_counter() - start, signature

/databricks/spark/python/pyspark/sql/readwriter.py in csv(self, path, schema, sep, encoding, quote, escape, comment, header, inferSchema, ignoreLeadingWhiteSpace, ignoreTrailingWhiteSpace, nullValue, nanValue, positiveInf, negativeInf, dateFormat

### Step 1: Create the Features

Using `bostonDF`, use a `VectorAssembler` object `assembler` to create a new column `newFeatures` that has the following three variables:<br><br>

1. `indus`: proportion of non-retail business acres per town
2. `age`: proportion of owner-occupied units built prior to 1940
3. `dis`: weighted distances to five Boston employment centers

Save the results to `bostonFeaturizedDF2`

In [0]:
# TODO
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=['indus', 'age', 'dis'], outputCol='newFeatures')
bostonFeaturizedDF2 = assembler.transform(bostonDF)

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-510601638879696> in <cell line: 5>()
      3 
      4 assembler = VectorAssembler(inputCols=['indus', 'age', 'dis'], outputCol='newFeatures')
----> 5 bostonFeaturizedDF2 = assembler.transform(bostonDF)

NameError: name 'bostonDF' is not defined

In [0]:
# TEST - Run this cell to test your solution
dbTest("ML1-P-02-01-01", True, set(assembler.getInputCols()) == {'indus', 'age', 'dis'})
dbTest("ML1-P-02-01-02", True, bool(bostonFeaturizedDF2.schema['newFeatures'].dataType))

print("Tests passed!")

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-510601638879697> in <cell line: 2>()
      1 # TEST - Run this cell to test your solution
----> 2 dbTest("ML1-P-02-01-01", True, set(assembler.getInputCols()) == {'indus', 'age', 'dis'})
      3 dbTest("ML1-P-02-01-02", True, bool(bostonFeaturizedDF2.schema['newFeatures'].dataType))
      4 
      5 print("Tests passed!")

NameError: name 'dbTest' is not defined

### Step 2: Train the Model

Instantiate a linear regression model `lrNewFeatures`.  Save the trained model to `lrModelNew`.

In [0]:
# TODO
from pyspark.ml.regression import LinearRegression

lrNewFeatures = LinearRegression(featuresCol='newFeatures', labelCol='medv')
lrModelNew = lrNewFeatures.fit(bostonFeaturizedDF2)

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-510601638879699> in <cell line: 5>()
      3 
      4 lrNewFeatures = LinearRegression(featuresCol='newFeatures', labelCol='medv')
----> 5 lrModelNew = lrNewFeatures.fit(bostonFeaturizedDF2)

NameError: name 'bostonFeaturizedDF2' is not defined

In [0]:
# TEST - Run this cell to test your solution
dbTest("ML1-P-02-02-01", True, lrNewFeatures.getFeaturesCol() == "newFeatures")
dbTest("ML1-P-02-02-02", True, lrNewFeatures.getLabelCol() == "medv")
dbTest("ML1-P-02-02-03", True, lrModelNew.hasSummary)

print("Tests passed!")

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-510601638879700> in <cell line: 2>()
      1 # TEST - Run this cell to test your solution
----> 2 dbTest("ML1-P-02-02-01", True, lrNewFeatures.getFeaturesCol() == "newFeatures")
      3 dbTest("ML1-P-02-02-02", True, lrNewFeatures.getLabelCol() == "medv")
      4 dbTest("ML1-P-02-02-03", True, lrModelNew.hasSummary)
      5 

NameError: name 'dbTest' is not defined

### Step 3: Create Predictions

Create the DataFrame `predictionsDF` for the following values, created for you in `newDataDF`:

| Feature | Datapoint 1 | Datapoint 2 | Datapoint 3 |
|:--------|:------------|:------------|:------------|
| `indus` | 11          | 6           | 19          |
| `age`   | 68          | 35          | 74          |
| `dis`   | 4           | 2           | 8           |

In [0]:
# TODO
from pyspark.ml.linalg import Vectors

data = [(Vectors.dense([11., 68., 4.]), ),
        (Vectors.dense([6., 35., 2.]), ),
        (Vectors.dense([19., 74., 8.]), )]
newDataDF = spark.createDataFrame(data, ["newFeatures"])
predictionsDF = lrModelNew.transform(newDataDF)

display(predictionsDF)

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-510601638879702> in <cell line: 8>()
      6         (Vectors.dense([19., 74., 8.]), )]
      7 newDataDF = spark.createDataFrame(data, ["newFeatures"])
----> 8 predictionsDF = lrModelNew.transform(newDataDF)
      9 
     10 display(predictionsDF)

NameError: name 'lrModelNew' is not defined

In [0]:
# TEST - Run this cell to test your solution
predicitions = [i.prediction for i in predictionsDF.select("prediction").collect()]

dbTest("ML1-P-02-02-01", True, predicitions[0] > 20 and predicitions[0] < 23)
dbTest("ML1-P-02-02-01", True, predicitions[1] > 30 and predicitions[1] < 34)
dbTest("ML1-P-02-02-01", True, predicitions[2] > 7 and predicitions[2] < 11)

print("Tests passed!")

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-510601638879703> in <cell line: 2>()
      1 # TEST - Run this cell to test your solution
----> 2 predicitions = [i.prediction for i in predictionsDF.select("prediction").collect()]
      3 
      4 dbTest("ML1-P-02-02-01", True, predicitions[0] > 20 and predicitions[0] < 23)
      5 dbTest("ML1-P-02-02-01", True, predicitions[1] > 30 and predicitions[1] < 34)

NameError: name 'predictionsDF' is not defined